In [735]:
import os
from openai import OpenAI

client = OpenAI(
    base_url="https://api.studio.nebius.ai/v1/",
    api_key=os.getenv('NB_STUDIO_API_KEY', ''),
)


In [737]:
batch_input_file = client.files.create(
  file=open("/Users/sofrony/batch.jsonl", "rb"),
  purpose="batch"
)

batch_input_file.dict()


{'id': 'd8af680e-9be8-4634-b4a5-cfd675067aef',
 'bytes': 9120315571,
 'created_at': 1729516591,
 'filename': 'batch.jsonl',
 'object': 'file',
 'purpose': 'batch',
 'status': None,
 'status_details': None}

In [738]:
batch_input_file_id = batch_input_file.id
print(batch_input_file_id)

batch_id = client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "nightly eval job"
    }
)

batch_id.to_dict()

d8af680e-9be8-4634-b4a5-cfd675067aef


{'id': 'batch_3416ef9e-7176-4ba1-8b00-8080984e0970',
 'completion_window': '24h',
 'created_at': 1729516667,
 'endpoint': '/v1/chat/completions',
 'input_file_id': 'd8af680e-9be8-4634-b4a5-cfd675067aef',
 'object': 'batch',
 'status': 'validating',
 'request_counts': {}}

In [739]:

status = client.batches.retrieve(batch_id.id)
status.to_dict()




{'id': 'batch_3416ef9e-7176-4ba1-8b00-8080984e0970',
 'completion_window': '24h',
 'created_at': 1729516667,
 'endpoint': '/v1/chat/completions',
 'input_file_id': 'd8af680e-9be8-4634-b4a5-cfd675067aef',
 'object': 'batch',
 'status': 'validating',
 'completed_at': None,
 'error_file_id': None,
 'finalizing_at': None,
 'in_progress_at': None,
 'output_file_id': None,
 'request_counts': {}}

In [753]:
batch_id = client.batches.list().dict()['data'][0]['id']
status = client.batches.retrieve(batch_id)
status.to_dict()

{'id': 'batch_df3346b5-4c2c-4f36-9b30-aeedfa485a59',
 'completion_window': '24h',
 'created_at': 1729502538,
 'endpoint': '/v1/chat/completions',
 'input_file_id': '6425e0cf-1f38-48c0-882b-c34336112cc0',
 'object': 'batch',
 'status': 'done',
 'completed_at': 1729513364,
 'error_file_id': None,
 'finalizing_at': 1729513363,
 'in_progress_at': 1729502547,
 'output_file_id': 'f01cbeef-4714-4edb-8896-25b712d287db',
 'request_counts': {'completed': 4, 'failed': 0, 'total': 4}}

In [754]:
file_response = client.files.retrieve(status.output_file_id)
file_response.dict()

{'id': 'f01cbeef-4714-4edb-8896-25b712d287db',
 'bytes': 2477,
 'created_at': 1729513364,
 'filename': 'batch_df3346b5-4c2c-4f36-9b30-aeedfa485a59_output.jsonl',
 'object': 'file',
 'purpose': 'batch',
 'status': None,
 'status_details': None}

In [755]:
import json
file_response = client.files.content(status.output_file_id)
lines = file_response.content.splitlines()
for line in lines:
    print(json.loads(line))

{'id': 'batch_req_e3f24ba0-8305-4395-9d1f-95703a363986', 'custom_id': 'request-1', 'response': {'id': 'chat-1bd17149372044158cc19d7859a5506d', 'choices': [{'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'message': {'content': 'Hello! How can I assist you today?', 'role': 'assistant', 'function_call': None, 'tool_calls': []}, 'stop_reason': None}], 'created': 1729513357, 'model': 'meta-llama/Meta-Llama-3.1-8B-Instruct-fast', 'object': 'chat.completion', 'service_tier': None, 'system_fingerprint': None, 'usage': {'completion_tokens': 10, 'prompt_tokens': 22, 'total_tokens': 32}, 'prompt_logprobs': None}, 'error': None}
{'id': 'batch_req_ded9f16e-af25-466b-a155-27a3a72dc09e', 'custom_id': 'request-2', 'response': {'id': 'chat-41d9be70810441c68b2fcd89b7f3bc58', 'choices': [{'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'message': {'content': '*shrugs*', 'role': 'assistant', 'function_call': None, 'tool_calls': []}, 'stop_reason': None}], 'created': 1729513359, 'model': 'me